In [19]:
import requests
import json
from typing import Dict, Any


def llm(
    messages: str,
    format_schema: Dict[str, Any] = None,
    api: str = 'https://ollama.selab.edu.vn',
    endpoint: str = '/api/chat',
    model: str = "qwen2.5:14b"
) -> Dict[str, str]:
    
    headers = {
      "Content-Type": "application/json"
    }


    if endpoint == '/api/chat' and format_schema is not None:
        data = {
            "model": model,
            "messages": messages,
            "format": format_schema,
            "stream": False
        }
    
    elif endpoint == '/api/generate':
        data = {
            "model": model,
            "prompt": messages,
            "option": {
              "temperature": 0.4  
            },
            "stream": False
        }

    url = f'{api}{endpoint}'
    
    response = requests.post(url, headers=headers, json=data, timeout=600)


    if response.status_code != 200:
        raise Exception(f"Request failed with status {response.status_code}: {response.text}")

    
    return response

In [26]:
import json
from typing import Dict

def greeting_checker_user_intent(query: str) -> str:
    user_prompt = f"""

    # ***User's query***
    {query}
    
    # General conversation guidelines:
    - Your task is to check if the user's query is a greeting conversation or not.
    - The user's query must be greeting conversation when the user's query is not related to any report question.
    - If the user's query is a greeting conversation, return 1.
    - If the user's query is not a greeting conversation, return 0.
    - The response should be in JSON format.
    

    # ***Example Scenarios:***
    
    - ***User***: "Hello bot how are you today ?"
    - ***Assistant***: {{"is_normal_conversation": 1}}
    
    - ***User***: "See you later. Bye."
    - ***Assistant***: {{"is_normal_conversation": 1}}

    - ***User***: "I want change to a little bit, I want to get Product Virtual Sports and product detail Saba Basketball with user level Super Agent"
    - ***Assistant***: {{"is_normal_conversation": 0}}

    - ***User***: "I want to get Winlost Report"
    - ***Assistant***: {{"is_normal_conversation": 0}}

    - ***User***: "Hey what is the weather in Tokyo?"
    - ***Assistant***: {{"is_normal_conversation": 1}}
    """
    return user_prompt


def get_decision(
    query: str
) -> Dict[str, str]:
    system_prompt = """
    You are a helpful assistant that can check if the user's query is a greeting conversation or not.
    
    # General conversation guidelines:
    - The user's query must be greeting conversation when the user's query is not related to any report question.
    - If the user's query is a greeting conversation, return 1.
    - If the user's query is not a greeting conversation, return 0.
    - The response should be in JSON format.
    """

    user_prompt = greeting_checker_user_intent(query)

    format_schema = {
        "type": "object",
        "properties": {
            "is_normal_conversation": {
                "type": "integer"
            }
        },
        "required": [
            "is_normal_conversation"
        ]
    }

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    response = llm(
        messages=messages,
        format_schema=format_schema
    )

    return response

In [37]:
response = get_decision("Oke bot let's do it")

In [38]:
json.loads(response.json()['message']['content'])['is_normal_conversation']

1